In [1]:
# Loading the data

In [2]:
import pandas as pd
df_graph = pd.read_csv('df_graph_with_wikidata.tsv',sep='\t')

In [3]:
df_graph.head()

,person1,person2,document_ids,wikidata_id1,wikidata_description1,wikidata_id2,wikidata_description2,weight,countries1,countries2,main_country1,main_country2
0,Sybren Valkema,Wim Crouwel,['0001246763-0001246764-0001246765-0001246766-...,Q2618110,Dutch glass artist (1916-1996),Q555389,"Dutch graphic designer, type designer, and typ...",4,['Kingdom of the Netherlands'],['Kingdom of the Netherlands'],Kingdom of the Netherlands,Kingdom of the Netherlands
1,Bert Haanstra,Sybren Valkema,['0001246763-0001246764-0001246765-0001246766-...,Q827371,Dutch film director (1916-1997),Q2618110,Dutch glass artist (1916-1996),10,['Kingdom of the Netherlands'],['Kingdom of the Netherlands'],Kingdom of the Netherlands,Kingdom of the Netherlands
2,Jane Bruce,Sybren Valkema,['0001246763-0001246764-0001246765-0001246766-...,Q16031419,Canadian teacher,Q2618110,Dutch glass artist (1916-1996),18,[],['Kingdom of the Netherlands'],NaN,Kingdom of the Netherlands
3,Frits Gramberg,Sybren Valkema,['0001246763-0001246764-0001246765-0001246766-...,Q42429178,Description not found.,Q2618110,Dutch glass artist (1916-1996),19,['Kingdom of the Netherlands'],['Kingdom of the Netherlands'],Kingdom of the Netherlands,Kingdom of the Netherlands
4,Bert Haanstra,Wim Crouwel,['0001246763-0001246764-0001246765-0001246766-...,Q827371,Dutch film director (1916-1997),Q555389,"Dutch graphic designer, type designer, and typ...",1,['Kingdom of the Netherlands'],['Kingdom of the Netherlands'],Kingdom of the Netherlands,Kingdom of the Netherlands


In [4]:
%
# Initialising weights

df_graph['weight'] = df_graph['document_ids'].map(lambda x: len(eval(x)))
df_graph.head()

UsageError: Line magic function `%` not found.


In [5]:
%
# Let's paint Americans one colour and Europeans another


import requests
from functools import lru_cache

@lru_cache(maxsize=1024)
def get_country_of_citizenship(entity_id):
    """
    Fetch the country of citizenship for a given Wikidata entity.

    Args:
    entity_id (str): The Wikidata entity ID (e.g., "Q42" for Douglas Adams).

    Returns:
    list: A list of country names for the entity's citizenship.
    """
    url = f"https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbgetentities",
        "ids": entity_id,
        "props": "claims",
        "format": "json"
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    countries = []
    
    # Extract the claims related to country of citizenship (P27)
    claims = data.get('entities', {}).get(entity_id, {}).get('claims', {})
    citizenship_claims = claims.get('P27', [])
    
    for claim in citizenship_claims:
        country_id = claim.get('mainsnak', {}).get('datavalue', {}).get('value', {}).get('id')
        if country_id:
            country_name = get_country_label(country_id)
            countries.append(country_name)
    
    return countries

@lru_cache(maxsize=128)
def get_country_label(country_id):
    """
    Fetch the label of a Wikidata entity given its ID.

    Args:
    country_id (str): The Wikidata country ID (e.g., "Q30" for the United States).

    Returns:
    str: The label (name) of the country.
    """
    url = f"https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbgetentities",
        "ids": country_id,
        "props": "labels",
        "languages": "en",
        "format": "json"
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    label = data.get('entities', {}).get(country_id, {}).get('labels', {}).get('en', {}).get('value', '')
    return label

# Example usage
entity_id = "Q42"  # Wikidata ID for Douglas Adams
countries = get_country_of_citizenship(entity_id)
print(f"Countries of citizenship for entity {entity_id}: {countries}")


UsageError: Line magic function `%` not found.


In [6]:
%
from tqdm.auto import tqdm
tqdm.pandas()
df_graph['countries1'] = df_graph['wikidata_id1'].progress_map(lambda x: get_country_of_citizenship(x))
df_graph['countries2'] = df_graph['wikidata_id2'].progress_map(lambda x: get_country_of_citizenship(x))

df_graph['main_country1'] = df_graph['countries1'].map(lambda x: x[0] if x else '')
df_graph['main_country2'] = df_graph['countries2'].map(lambda x: x[0] if x else '')

df_graph.to_csv('df_graph_with_wikidata.tsv',sep='\t',index=False)
df_graph.head()

UsageError: Line magic function `%` not found.


In [7]:
all_countries = set(df_graph['main_country1'].tolist()+df_graph['main_country1'].tolist())
all_countries

{'Australia',
 'Austria',
 'Austria-Hungary',
 'Belgium',
 'Brazil',
 'Canada',
 'Cisleithania',
 'Czech Republic',
 'Czechoslovakia',
 'Denmark',
 'Dutch East Indies',
 'Dutch Republic',
 'England',
 'Finland',
 'France',
 'German Democratic Republic',
 'German Empire',
 'German Reich',
 'Germany',
 'Indonesia',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Kingdom of Denmark',
 'Kingdom of Italy',
 'Kingdom of the Netherlands',
 'Lithuania',
 'Luxembourg',
 'Monaco',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Poland',
 'Portugal',
 'Republic of Ireland',
 'Slovakia',
 'South Africa',
 'Soviet Union',
 'Spain',
 'Sweden',
 'Switzerland',
 'United Kingdom',
 'United Kingdom of Great Britain and Ireland',
 'United States of America',
 'Uruguay',
 nan}

In [8]:
from tqdm.auto import tqdm
country_by_person = {}
for _,row in tqdm(df_graph.iterrows()):
    country_by_person[row['person1']] =row['main_country1']
    country_by_person[row['person2']] =row['main_country2']

0it [00:00, ?it/s]

In [9]:
import pickle
pickle.dump(country_by_person,open('country_per_person.p','wb'))

In [ ]:
# Downsample the graph to deal with computational limitations: only keep the edges that are heavy enough

In [ ]:
df_graph_common = df_graph[df_graph['weight']>=10]

In [ ]:
df_graph_common

In [ ]:
colours = {'United States of America':'#1E90FF',#blue
           'Kingdom of the Netherlands': '#FFA500',#orange
          }

def get_colour(person):
    country = country_by_person[person] if person in country_by_person else ''
    if country in colours:
        return colours[country]
    return '#9370DB' # purple


# def hex_to_rgb(hex_color):
#     """Convert hex color to RGB tuple."""
#     hex_color = hex_color.lstrip('#')  # Remove the leading '#' if present
#     return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# def rgb_to_hex(rgb_color):
#     """Convert RGB tuple to hex color."""
#     return '#{:02x}{:02x}{:02x}'.format(rgb_color[0], rgb_color[1], rgb_color[2])

# def average_hex_colors(hex_color1, hex_color2):
#     """Average two hex colors and return the resulting hex color."""
#     # Convert hex to RGB
#     rgb1 = hex_to_rgb(hex_color1)
#     rgb2 = hex_to_rgb(hex_color2)
    
#     # Calculate the average of each RGB component
#     avg_rgb = tuple((int((comp1 + comp2) / 2)) for comp1, comp2 in zip(rgb1, rgb2))
    
#     # Convert the averaged RGB back to hex
#     return rgb_to_hex(avg_rgb)

In [ ]:
import networkx as nx


In [ ]:
G = nx.Graph()
cnt = 0
for _,row in df_graph_common.iterrows():
    cnt += 1
    G.add_edge(row['person1'],row['person2'],weight=row['weight']/20)
#     if cnt == 350:
#         break

In [ ]:
from pyvis.network import Network
import numpy as np
def prepare_network(G1):
    # Remove components that are too small (for visualisation purposes)
    for component in list(nx.connected_components(G1)):
        if len(component)<3:
            for node in component:
                G1.remove_node(node)
    # Calculate node degrees
    node_degrees = dict(G1.degree())
    
    net = Network(notebook=True,cdn_resources='remote',bgcolor='#f5f5f5',height='5500px',width='5500px')
    
    # Define a scaling factor for font size (adjust as needed)
    font_scale = 5
    node_scale = 5#0

    # Add nodes with sizes and font sizes proportional to their degrees
    for node_id, degree in node_degrees.items():
        node_size = np.log(degree)*node_scale#np.log2(degree) * node_scale # Scale node size (adjust as needed)
        font_size = np.sqrt(degree)*font_scale#degree*font_scale#np.log(degree)*font_scale# * font_scale  # Scale font size
        colour = get_colour(node_id)
        net.add_node(node_id, size=node_size, label=str(node_id),color=colour,
                     shape='dot',
                     font={'size': font_size,'color': 'black'},#colour},
                    labelHighlightBold=True)

    # Add edges with weights
    for u, v, data in G.edges(data=True):
        weight = data['weight']  # Extract the weight of the edge
        net.add_edge(u, v, value=weight,
                     color='#d6ebc3')#average_hex_colors(get_colour(u),get_colour(v)))  # Use 'value' to set the edge weight in Pyvis
        
    # Configure physics for repulsion
#     net.hrepulsion(
#         node_distance=350,    # Distance between nodes
#         central_gravity=3.5, # Strength of the central gravity
#         spring_length=100,    # Length of the springs
#         spring_strength=0.035, # Strength of the springs
#         damping=0.6#,          # Damping factor to stabilize the layout
# #         -=1
#     )
#     net.force_atlas_2based(gravity= -950,central_gravity = 0,overlap = 1)
        
    net.repulsion(
        node_distance=450,    # Distance between nodes
        central_gravity=0.05, # Strength of the central gravity
        spring_length=350,    # Length of the springs
        spring_strength=0.05, # Strength of the springs
        damping=0.6#,          # Damping factor to stabilize the layout
#         avoid_overlap=1
    )

    net.toggle_physics(True)
#     net.set_options(options)

#     net.from_nx(G1)
    
    return net

In [ ]:
net = prepare_network(G)
# net.repulsion()
net.show_buttons(filter_=["physics"])#,"autoResize","layout"])
net.show('graph-wikidata-final-big.html')
# net.write_graphml('valkema.graphml')


In [ ]:
#  net.hierarchical_repulsion(
#         node_distance=750,    # Distance between nodes
#         central_gravity=0.05, # Strength of the central gravity
#         spring_length=150,    # Length of the springs
#         spring_strength=0.05, # Strength of the springs
#         damping=0.6,          # Damping factor to stabilize the layout
#         avoid_overlap=1
#     )